# G1 Baseline
Creating a baseline model for the MSMARCO TREC CAsT dataset using PyTerrier with a basic BM25 model and Random Forest for reranking.

In [44]:
import pandas as pd
import pyterrier as pt
import re

from pyterrier.measures import *
from sklearn.ensemble import RandomForestRegressor

if not pt.started():
  pt.init()

## Download and extract dataset

The *MS MARCO Passages* dataset was downloaded from https://gustav1.ux.uis.no/dat640/msmarco-passage.tar.gz and unpacked in the folder named `data`. Note that this is only needed to do once, so the code has been commented out to avoid running it unnecessarily.

In [7]:
# !mkdir -p data && \
#   cd data && \
#   wget -nc https://gustav1.ux.uis.no/dat640/msmarco-passage.tar.gz && \
#   tar -xzf msmarco-passage.tar.gz

File ‘msmarco-passage.tar.gz’ already there; not retrieving.



## Building the index
This is only needed to initially create the index from the collection corpus. It's also possible to download pre-made indices (http://data.terrier.org/msmarco_passage.dataset.html). Since the dataset was provided from a different source than the official there may be some differences, so we decided to build the index ourself on the actual downloaded dataset. Note that it took about 20 minutes to generate the index on a fast workstation (Intel i9 3.7 GHz CPU, 32 GB RAM, and a 3500 MB/s NVMe SSD disk). It could be an advantage to change to Elasticsearch for building and working with the index, especially for large corpus.

Ref. https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html#iterdictindexer

In [4]:
dataset_path = './data/collection.tsv'


def msmarco_generate():
    with pt.io.autoopen(dataset_path, 'rt') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno': docno, 'text': passage}


iter_indexer = pt.IterDictIndexer(
    "./index", blocks=True, overwrite=False, verbose=True, meta={'docno': 20, 'text': 4096})

indexref = iter_indexer.index(msmarco_generate())

18:37:34.123 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (500080) - further warnings are suppressed
18:56:46.730 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer - Indexed 5 empty documents


## Accessing the index

Here we establish a pointer to the generated index.

Ref. https://github.com/terrier-org/pyterrier/blob/master/examples/notebooks/retrieval_and_evaluation.ipynb

In [18]:
index = pt.IndexFactory.of('./index')
print(index.getCollectionStatistics().toString())

Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   true



## Read topics
For the baseline model we use the raw queries without any topic rewriting or context history. Only the most basic cleaning has been performed, by removing any special characters and lowercasing everything.

In [9]:
topics = pd.read_csv('./data/queries_train.csv', usecols=['qid', 'query'])
# Remove special characters
topics['query'] = topics['query'].apply(lambda x: re.sub(r'\W', ' ', x).lower())
topics

,qid,query
0,4_1,what was the neolithic revolution
1,4_2,when did it start and end
2,4_3,why did it start
3,4_4,what did the neolithic invent
4,4_5,what tools were used
...,...,...
248,105_5,who named the movement
249,105_6,what was the us reaction to it
250,105_7,tell me more about the movement of the police ...
251,105_8,why were they killed


## Read qrels from file
Here we read the query relevances that was provided for the training data.

In [35]:
# Load query relevance for the training set
qrels = pd.read_csv('./data/qrels_train.txt', sep=' ', names=['qid', 'iter', 'docno', 'label'])
qrels['docno'] = qrels['docno'].astype(str)
qrels

,qid,iter,docno,label
0,4_1,0,2253187,1
1,4_1,0,813726,2
2,4_1,0,813729,2
3,4_1,0,2253186,2
4,4_1,0,5414512,0
...,...,...,...,...
17059,105_9,0,7853976,0
17060,105_9,0,7985635,0
17061,105_9,0,801480,3
17062,105_9,0,801482,1


## Running experiments  

In this section we read the training set queries, setup the PyTerrier pipeline, and run BatchRetrival where we return the top 1000 results for each query. The results are finally saved in TREC format.

refs.:
* https://pyterrier.readthedocs.io/en/latest/terrier-retrieval.html
* https://github.com/terrier-org/pyterrier/blob/6698e36f24e02ff3725247e735b791237755085d/examples/experiments/Robust04.ipynb
* https://github.com/terrier-org/pyterrier/blob/master/examples/notebooks/retrieval_and_evaluation.ipynb

Simple weighting models:

In [73]:
BM25 = pt.BatchRetrieve(index, num_results=1000, verbose=True, wmodel="BM25")
DPH  = pt.BatchRetrieve(index, num_results=1000, verbose=True, wmodel="DPH")
PL2  = pt.BatchRetrieve(index, num_results=1000, verbose=True, wmodel="PL2")
DLM  = pt.BatchRetrieve(index, num_results=1000, verbose=True, wmodel="DirichletLM")

In [75]:
# Run experiment with plain BM25 model
exp_simple_wm = pt.pipelines.Experiment(
    [BM25, DPH, PL2, DLM], 
    topics, 
    qrels, 
    eval_metrics=[R(rel=2)@1000, nDCG@3, AP(rel=2), RR(rel=2)],
    names=["BM25", "DPH", "PL2", "Dirichlet QL"], 
    save_dir='./experiments',
    save_mode='overwrite',
    filter_by_topics=False,
    )

BR(DirichletLM): 100%|██████████| 253/253 [01:14<00:00,  3.38q/s]


In [78]:
# Show results
exp_simple_wm

,name,R(rel=2)@1000,nDCG@3,AP(rel=2),RR(rel=2)
0,BM25,0.383571,0.095441,0.071540,0.139359
1,DPH,0.395657,0.097868,0.072007,0.137880
2,PL2,0.378016,0.095977,0.071830,0.137782
3,Dirichlet QL,0.365456,0.085548,0.074656,0.118252


### Reranking using Random Forest and additional features

Here we try to improve the retrieval results by using a Random Forest reranker and additional features from the TF_IDF, PL2, and DLM models.

Refs.:
* https://pyterrier.readthedocs.io/en/latest/ltr.html
* https://github.com/terrier-org/pyterrier/blob/master/examples/notebooks/ltr.ipynb (seems to be outdated, pt.pipelines.LTR_pipeline no longer exists)


In [97]:
# Define models
RFR = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
TF_IDF =  pt.BatchRetrieve(index, num_results=1000, verbose=True, wmodel='TF_IDF')

In [98]:
# Define pipeline and fit reranking model
pipe = BM25 >> (pt.transformer.IdentityTransformer() ** TF_IDF ** PL2 ** DLM) >> pt.ltr.apply_learned_model(RFR)
pipe.fit(topics, qrels)

BR(DirichletLM): 100%|██████████| 252/252 [01:04<00:00,  3.89q/s]
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:   26.1s finished


In [99]:
# Run experiment
exp_rf_rerank = pt.pipelines.Experiment(
    [pipe],
    topics,
    qrels,
    eval_metrics=[R(rel=2)@1000, nDCG@3, AP(rel=2), RR(rel=2)],
    names=['BM25_TFIDF_PL2_DLM_RF'],
    save_dir='./experiments',
    save_mode='overwrite',
    filter_by_topics=False,
)

BR(DirichletLM): 100%|██████████| 252/252 [01:04<00:00,  3.93q/s]
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 400 out of 400 | elapsed:    0.4s finished


In [100]:
# Show results
exp_rf_rerank

,name,R(rel=2)@1000,nDCG@3,AP(rel=2),RR(rel=2)
0,BM25_TFIDF_PL2_DLM_RF,0.383571,0.407454,0.306488,0.435386


## Ranking test queries
Using the best baseline model to rank the test queries for posting to the Kaggle competition.

In [101]:
topics_test = pd.read_csv('./data/queries_test.csv', usecols=['qid', 'query'])
# Remove special characters
topics_test['query'] = topics_test['query'].apply(lambda x: re.sub(r'\W', ' ', x).lower())
topics_test

,qid,query
0,1_1,what is a physician s assistant
1,1_2,what are the educational requirements required...
2,1_3,what does it cost
3,1_4,what s the average starting salary in the uk
4,1_5,what about in the us
...,...,...
243,102_5,how much is owed
244,102_6,when will it run out of money
245,102_7,wow what will happen
246,102_8,can it be fixed


Run batch retrieval, scoring based on nDCG@3. Only the three top-ranked document ID's required for each topic.

In [153]:
pipe2 = pipe % 3
queries_test_results = pipe2.transform(topics_test)

BR(DirichletLM): 100%|██████████| 245/245 [00:58<00:00,  4.19q/s]
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 400 out of 400 | elapsed:    0.4s finished


In [154]:
queries_test_results[['topic_number', 'turn_number']] = queries_test_results['qid'].str.split('_', expand=True)
test_res = queries_test_results.sort_values(['topic_number', 'turn_number', 'score'], ascending=[True, True, False])
test_res[['qid', 'docid']]

,qid,docid
7,1_1,5780724
8,1_1,3951096
19,1_1,2329378
8193,1_10,1244400
8074,1_10,1852633
...,...,...
223126,99_7,438713
223698,99_7,3180972
224154,99_8,3940798
224044,99_8,290622


Save to CSV file with headings qid,docid

In [155]:
test_res[['qid', 'docid']].to_csv('./results/queries_test_results.csv', index=False)